In [3]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Question:  By observing how certain types of crime cluster around certain addresses, given an address, can we predict 
# what type of crime is being called in with a reasonable degree of accuracy? We plan to build a k-NN classifier which 
# takes in a police report coming in from a certain address and is able to predict accurately what crime is taking place. 
# We will use training and testing sets to calibrate the classifier in order to improve accuracy in this question. 

# The following code will use the modified crimes table from the arrest rates question, where the crime categories have been 
# sorted into the broader categories of nonviolent misdemeanors, etc. The procedure here is to use our training set to find the 
# right k value for which k-NN yields the highest degree of accuracy. 

# The modified crimes table from the arrest rates question, where the crime categories have been sorted into the broader 
# categories of nonviolent misdemeanors, etc. We'll be working with a downsized sample of this table for faster processing.

crimes = Table.read_table('crimes_for_classifier.csv').sample(10000)

In [4]:
# In this cell, we split the crimes table into a training set and a test set. The test set will be put aside for later, when we
# use it to evaluate the true accuracy of our final classifier. 
training_proportion = 17/20
rows = crimes.num_rows
num_train = int(rows * training_proportion)

train_set = crimes.take(np.arange(num_train))
test_set = crimes.take(np.arange(num_train, rows))

In [5]:
# These two functions make up the basis of our classifier. For a given crime report, the classify function finds its distance to 
# every other crime report in the training set by using longitude/latitude distance, and sorts them from shortest distance
# to longest. The function then takes the top k neighbors with the shortest distances, and spits back out the crime category 
# which is the majority of those neighbors as a prediction of the original crime report.

def distance_two_features(row0, row1):
    return np.sqrt((row0.item('X') - row1.item('X'))**2 + (row0.item('Y') - row1.item('Y'))**2)

def classify(arbitrary_row_number):
    distances = make_array()
    new_train_set = train_set.exclude(arbitrary_row_number)
    for i in new_train_set.rows:
        distances = np.append(distances, distance_two_features(train_set.row(arbitrary_row_number), i))
    with_distance = new_train_set.with_column("Distance", distances).sort('Distance').take(np.arange(3)).group('Category').sort('count', descending=True)
    return with_distance.column(0).item(0)


In [6]:
# Example: Classify would take in some row number of the training set, and return a prediction
classify(0)

'ASSAULT'

In [ ]:
# This part of the code generates predictions for the entire training set. Code takes a very, very long time. 
collector = make_array()
for i in np.arange(train_set.num_rows):
    collector = np.append(collector, classify(i))
train_set_with_predictions = train_set.with_column('Predictions', collector)

In [ ]:
# Example of what the training set with predictions looks like
train_set_with_predictions.show(10)

In [ ]:
# This returns the proportion for which the predictions match the actual category of the crime in the training set. This tells 
# us how accurate our classifier is for the training set. 
proportion_correct = np.count_nonzero(train_set_with_predictions.column('Predictions') == train_set_with_predictions.column('Category')) / train_set_with_predictions.num_rows
proportion_correct

In [ ]:
# Above, we arbitrarily chose 3 as a k-value. After experimenting with other k-values, we found that k = 7 seemed to yield the 
# highest accuracy. We thus test it on the test set, after adjusting our classifier function.

def classify_test(arbitrary_row_number):
    distances = make_array()
    for i in train_set.rows:
        distances = np.append(distances, distance_two_features(test_set.row(arbitrary_row_number), i))
    with_distance = train_set.with_column("Distance", distances).sort('Distance').take(np.arange(7)).group('Category').sort('count', descending=True)
    return with_distance.column(0).item(0)
                              
collector = make_array()
for i in np.arange(test_set.num_rows):
    collector = np.append(collector, classify_test(i))
test_set_predict = test_set.with_column('Predictions', collector)
                              
proportion_correct = np.count_nonzero(test_set_predict.column('Predictions') == test_set_predict.column('Category')) / test_set_predict.num_rows
proportion_correct

In [ ]:
# In experimenting with our k-values, we observed that the accuracy did not seem to improve much regardless of the k-value 
# used. We suggest this is because the data set itself resists easy classification by distance, as given types of crime do not 
# easily clump together into clusters by which nearest-neighbor classification would be most accurate. If we visualize the 
# distribution of just the top three types of crime for just a small area of SF, we can see how nearest-neighbor classification 
# would be limited in accuracy. 

visual = crimes.where('Category', are.not_equal_to('VEHICLE THEFT')).where('Category', are.not_equal_to('BURGLARY')).where('Category', are.not_equal_to('ROBBERY')).where('Category', are.not_equal_to('NONVIOLENT FELONIES')).where('Category', are.not_equal_to('AGGRAVATED MISDEMEANORS')).where('Category', are.not_equal_to('SEX OFFENSES, FORCIBLE')).where('Category', are.not_equal_to('ARSON'))
visual = new.where('X', are.between(-122.4, -122.3)).where('Y', are.between(37.75, 37.78))
type_to_color = Table().with_columns("Category", make_array('LARCENY/THEFT', 'NONVIOLENT MISDEMEANORS', 'ASSAULT'), "Color", make_array('gold', 'blue', 'green'))
with_colors = visual.join('Category', type_to_color)
with_colors.scatter("X", "Y", colors="Color")

# Blue is larceny/theft, gold is nonviolent misdemeanors, green is assault

Question answer: Using a k-NN classifier to predict crimes is possible to some degree of accuracy, but that accuracy is severely limited by the lack of distinct distribution of crime types in the dataset, and seems to cap out at under 40%. Thus, we find that a k-NN classifier is limited in its ability to predict crimes accurately given only an address. Given more data, perhaps average housing prices around that address or what time of day the call is made, we could include more characteristics in our classifier and build a more accurate predictor - given the constraints of the dataset however, this is likely around where a k-NN address classifier maxes out in accuracy.